# Plan and execute with tool - Execute

In [ ]:
from openai import OpenAI
from typing import List, Dict

def ask(messages: List[Dict], is_json:bool = False):
        client = OpenAI(api_key='YOUR DEEPSEEK KEY', base_url="https://api.deepseek.com")

        if is_json:
                response = client.chat.completions.create(
                        model = 'deepseek-chat',
                        temperature = 0,
                        messages = messages,
                        response_format={ "type": "json_object" }
                )
        else:
                response = client.chat.completions.create(
                        model = 'deepseek-chat',
                        temperature = 0,
                        messages = messages,
                )

        return response

In [ ]:
## Web Search
from search import internet_search, process_content

class WebSearch:
    def __init__(self, name:str='web_search', threhold:int=8000):
        self.system_prompt = """
You are a Insight Researcher.

1. To find detail informtion for the user query
and summary the content into one sentence as simple as possible
2. If the user's question is about specific numerical values, 
only return the numerical results without any additional explanation.
"""
        self.name = name
        self.description = "the tool use for web search"
        self.threhold = threhold

    def __call__(self, query:str):
        results = internet_search(query)
        # print(results)
        all_text = ""
        windows_size = 0
        for item in results:
            if windows_size >= self.threhold:
                break
            page_content = process_content(item['href'])
            for page in page_content:
                if windows_size + len(page) > self.threhold:
                    remaining_space = self.threhold - windows_size
                    all_text += page[:remaining_space].strip() + "\n\n"
                    windows_size = self.threhold
                    break
                else:
                    windows_size += len(page)
                    all_text += page + "\n\n"
            if windows_size >= self.threhold:
                break

        msg = [{"role":"system","content":self.system_prompt},
               {"role":"user", "content": f"The search query {query}\nThe search results are {all_text}"}]
        
        answer = ask(messages=msg)

        return answer.choices[0].message.content


In [ ]:
class AbsDifference:
    def __init__(self, name:str="subtractions"):
        self.system_prompt = "You are a calculation assistant."

        self.name = name
        self.description = "math tool for calculating subtractions"
        
    def __call__(self, question:str) -> str:

        user_content = f"""
Answer the Question based on the Context. When you write down a expression, it MUST ONLY consists of numbers and operators. Here are some guidelines that you will be PANALIZED if you don't follow:

  - When you are asked for differences, you consider the absolute value of the difference. Difference of two numbers is always positive.For instance, the difference between 1 and 2 is 1, not -1.
  - When you are applying operations (e.g. difference, summation, ratio, etc.) between multiple values in the Context, you must unify the units of those numbers. For instance, you cannot add 1 meter to 1 foot.
     - You must pick the values in the same units if all the values are available in the same units.
     - If not, you must convert them to the same units before applying the operation.
  - You MUST strictly follow the unit (e.g. meter, kilometer, million, etc.) you were asked.
     - If the Context has the numbers in same units as the question, you can directly use them.
     - If the Context has the numbers in different units than the question, you must convert them to the units asked in the question.For example, if the question asks for the distance between two cities in kilometers, but the Context has the distance in miles, you must convert the distance to kilometers.
  - If you are asked about a particular number in millions, billions, or any other unit, the number should be written without specifying the unit. For example, if you are asked for 100 millions, it should be written as 100, not 100 million or 100,000,000.
 - Never introduce a variable. For instance "gazelle_max_speed * 1.4" is not allowed. Pick up a correct number from the given context.

 Question: {question}
 
"""
        messages = [
            {"role":"system", "content":self.system_prompt},
            {"role":"user", "content":user_content}
        ]

        response = ask(messages=messages)
        return response.choices[0].message.content

In [ ]:
search_tool = WebSearch(name="search")
math_tool = AbsDifference(name="math")

tools = [search_tool, math_tool]

## Execute

In [ ]:
plans = """
Plan:
1. search('Toronto population 2023')
2. search('New York City population 2023')
3. math('population difference between New York City and Toronto', ['$1', '$2'])
Thought: I can answer the question now.
"""

In [ ]:
import re

ACTION_PATTERN = r"\n*(\d+)\. (\w+)\((.*)\)(\s*#\w+\n)?"
matches = re.findall(ACTION_PATTERN, plans)
print(matches)

In [ ]:
from typing import Any, Sequence, Union, Callable, Collection, Dict, List, Optional
class Step:
    def __init__(self, 
                 idx: int, 
                 name: str, 
                 tool: callable,
                 args: Collection[Any],
                 dependencies: Collection[int]):
        self.idx = idx
        self.name = name
        self.tool = tool
        self.args = args
        self.dependencies = dependencies
        self.observation = None

    def exec(self):
        self.observation = self.tool(self.args)
        return self.observation

In [ ]:
ID_PATTERN = r"\$\{?(\d+)\}?"

def default_dependency_rule(idx, args: str):
    matches = re.findall(ID_PATTERN, args)
    numbers = [int(match) for match in matches]
    return idx in numbers

In [ ]:
def find_tool(tool_name:str, tools: Sequence[Callable]=tools):
    for tool in tools:
        if tool.name == tool_name:
            return tool

In [ ]:
steps = []
for item in matches:
    idx, tool_name, tool_args, _ = item
    idx = int(idx)
    print(f"step {idx}")
    print(f"tool name is {tool_name}, args is {tool_args}")
    
    # match tool
    tool = find_tool(tool_name)

    # get dependencies
    dependencies = [i for i in range(1, idx) if default_dependency_rule(i, tool_args)]
    print(f"the dependencies are {dependencies}")

    # build step object
    step = Step(
        idx=idx,
        name=tool_name,
        tool=tool,
        args=tool_args,
        dependencies=dependencies
    )

    steps.append(step)

In [ ]:
step1 = steps[0]
print(f"the args of step1 {step1.args}")
result1 = step1.exec()
print("the result of step1", result1)

In [ ]:
step2 = steps[1]
print(f"the args of step2 {step2.args}")
result2 = step2.exec()
print("the result of step2", result2)

In [ ]:
step3 = steps[2]
for dependency in sorted(step3.dependencies, reverse=True):
    for arg_mask in ["${" + str(dependency) + "}", "$" + str(dependency)]:
        if arg_mask in step3.args:
            print(arg_mask)
            if steps[dependency-1].observation is not None:
                print(steps[dependency -1].observation)
                step3.args = step3.args.replace(
                                arg_mask, str(steps[dependency-1].observation)
                            )

In [ ]:
print(f"the args of step3 {step3.args}")

In [ ]:
result3 = step3.exec()
print(result3)